<a href="https://colab.research.google.com/github/trybalad/BERT/blob/master/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy
!python -m spacy download pl_core_news_lg

In [ ]:
#Jeśli szkolimy z gpu
!pip install tensorflow-gpu

In [ ]:
#Jeśli wykorzystujemy cpu
!pip install tensorflow

In [ ]:
from keras_bert.data_generator import DataGenerator
from keras_bert.model import create_model
from keras_bert.tokenizer import Tokenizer
from keras_bert.training import train_model

In [ ]:
#Reading vocab.
tokenizer = Tokenizer()
tokenizer.read_vocab('./data/counted_vocab.txt')
tokenizer.change_to_reversible()
print("Vocab of size:", tokenizer.vocab_size, "loaded.")

In [ ]:
max_len = 32
embedding_dim = 512
ff_dim = 512
heads = 4
encoder_num = 4

checkpoint_file_path = "./data/checkpoint_notebook.ckpt"
load_checkpoint = False

In [ ]:
#creating data generator
data_generator = DataGenerator("./data/corpus_clean.txt", max_len, tokenizer, batch_size=64, create_nsr_output=True)
print("Data generator prepared.")

In [ ]:
#preparing model
sequence_encoder = create_model(tokenizer.vocab_size, max_len, embedding_dim, encoder_num, heads, encoder_num)
print("Model created.")

In [ ]:
# Start training.
train = train_model(sequence_encoder, max_len, tokenizer, data_generator, epochs=100,
                    checkpoint_file_path=checkpoint_file_path, load_checkpoint=load_checkpoint)

In [ ]:
from keras_bert.prepare_data import create_tokens, create_masks, create_ids, create_segments, translate_ids, \
    create_pretrain_data
import numpy as np

In [ ]:
message = "Pojęcie w programowaniu obiektowym najczyściej utożsamiane z klasą. Jest modelem, który w rzeczywistości nie reprezentuje żadnego istniejącego obiektu, a jedynie na podstawie którego definiowane są inne obiekty."
tokens = create_tokens([message], tokenizer, max_len)
mask_t = create_pretrain_data(tokens, tokenizer)
ids = create_ids(mask_t, max_len, tokenizer)
mask = create_masks(mask_t, max_len)
segments = create_segments(mask_t, max_len)

result = model.predict(x = [np.array(ids), np.array(segments), np.array(mask)])
print(tokens)
print(mask_t)
print(ids)
print(translate_ids(result, tokenizer))